In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

In [2]:
print("[*] data preprocessing")

train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')


[*] data preprocessing


In [3]:
train_data = np.array(train_df.iloc[:,1:], dtype = 'float32')
test_data = np.array(test_df.iloc[:,1:], dtype='float32')
print(train_data)
print(test_data)

[[2. 0. 0. ... 0. 0. 0.]
 [9. 0. 0. ... 0. 0. 0.]
 [6. 0. 0. ... 0. 0. 0.]
 ...
 [8. 0. 0. ... 0. 0. 0.]
 [8. 0. 0. ... 0. 0. 0.]
 [7. 0. 0. ... 0. 0. 0.]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 1. 3. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [4]:
x_train = train_data[:,1:]/255
y_train = train_data[:,0]
x_test= test_data/255
print(x_train)
print("----------")
print(y_train)
print("-----------")
print(x_test)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
----------
[2. 9. 6. ... 8. 8. 7.]
-----------
[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.00392157 0.        ]
 [0.         0.00392157 0.01176471 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [5]:
x_train,x_validate,y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,
                                                         random_state = 12345)
print(x_train)
print(x_validate)
print(y_train)
print(y_validate)

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.08235294 0.03921569 0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
[6. 5. 9. ... 9. 7. 9.]
[7. 3. 3. ... 0. 7. 4.]


In [6]:
x_train = x_train.reshape(x_train.shape[0],28,28,1)
x_test = x_test.reshape(x_test.shape[0],28,28,1)
x_validate = x_validate.reshape(x_validate.shape[0],28,28,1)

print(x_train)
print(x_validate)
print(y_train)
print(y_validate)

[[[[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  ...

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]]


 [[[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  ...

  [[0.        ]
   [0.        ]
   [0. 

In [17]:
print("[*] model generation")
cnn_model = Sequential([
    Conv2D(32, kernel_size=3, activation='relu', padding='same', input_shape = (28,28,1)),
    MaxPooling2D(2),
    Conv2D(64, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Conv2D(128, kernel_size=3, activation='relu', padding='same'),
    MaxPooling2D(2),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(.4),
    Dense(10, activation='softmax'),
])



[*] model generation


In [18]:
print("[*] model training..")
cnn_model.compile(optimizer='NAdam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])



[*] model training..


In [19]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('best-cnn-model.h5')
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
history = cnn_model.fit(x_train,y_train, epochs=100, validation_data=(x_validate, y_validate), 
                    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
1500/1500 [==============================] - 33s 21ms/step - loss: 0.5555 - accuracy: 0.8001 - val_loss: 0.3406 - val_accuracy: 0.8751
Epoch 2/100
1500/1500 [==============================] - 32s 21ms/step - loss: 0.3503 - accuracy: 0.8743 - val_loss: 0.2835 - val_accuracy: 0.8959
Epoch 3/100
1500/1500 [==============================] - 32s 22ms/step - loss: 0.2996 - accuracy: 0.8929 - val_loss: 0.2615 - val_accuracy: 0.9024
Epoch 4/100
1500/1500 [==============================] - 31s 21ms/step - loss: 0.2632 - accuracy: 0.9062 - val_loss: 0.2421 - val_accuracy: 0.9111
Epoch 5/100
1500/1500 [==============================] - 31s 21ms/step - loss: 0.2377 - accuracy: 0.9143 - val_loss: 0.2257 - val_accuracy: 0.9170
Epoch 6/100
1500/1500 [==============================] - 32s 21ms/step - loss: 0.2152 - accuracy: 0.9216 - val_loss: 0.2463 - val_accuracy: 0.9165
Epoch 7/100
1500/1500 [==============================] - 32s 22ms/step - loss: 0.2002 - accuracy: 0.9283 - val_loss: 0

In [20]:
y_pred = cnn_model.predict(x_test)
y_pred = y_pred.argmax(axis=-1)


313/313 [==============================] - 2s 6ms/step


In [21]:
submission = pd.read_csv('./data/sample_submission.csv', encoding = 'utf-8')
submission['label'] = y_pred
submission.to_csv('fashion_submission.csv', index = False)